In [3]:
from pyspark.context import SparkContext
from pyspark import SparkFiles
from pyspark.sql.session import SparkSession
import os


spark = SparkSession.builder.master("local[*]") \
    .appName("S3 CSV Example") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .getOrCreate()

# spark = configure_spark_with_delta_pip(spark).getOrCreate()


spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", os.getenv('AWS_ACCESS_KEY_ID'))
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", os.getenv('AWS_SECRET_ACCESS_KEY'))
spark._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")


In [4]:
from dotenv import load_dotenv
import os
import boto3

In [5]:
# # Carregar variáveis de ambiente do arquivo .env
# load_dotenv()

# # Obter variáveis de ambiente
MINIO_ENDPOINT = 'http://minio:9000'
# MINIO_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
# MINIO_SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

In [6]:
s3 = boto3.client('s3', region_name='sa-east-1')  # Specify the region

In [7]:
# Exemplo de leitura/escrita no MinIO
bucket_name = "my-delta-lake-1"
s3_path = f"s3a://{bucket_name}/delta/sales_data"

try:
    response = s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': 'sa-east-1'}
    )
except:
    pass

In [8]:
data = [("James", "Smith", "USA", 1), ("Michael", "Rose", "USA", 2)]
columns = ["firstname", "lastname", "country", "id"]
df = spark.createDataFrame(data, schema=columns)

In [9]:
local_delta_path = "/home/jovyan/work/delta-table"
os.makedirs(local_delta_path, exist_ok=True)

In [10]:
def upload_directory_to_s3(local_directory, bucket, s3_directory):
    for root, dirs, files in os.walk(local_directory):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, local_directory)
            s3_key = os.path.join(s3_directory, relative_path).replace("\\", "/")
            s3.upload_file(local_path, bucket, s3_key)
            print(f"Uploaded {local_path} to s3://{bucket}/{s3_key}")

In [11]:
df.write.csv(s3_path, header=True, mode='overwrite')

In [12]:
# Leia o CSV do S3 em um DataFrame
df_from_s3 = spark.read.csv(s3_path, header=True, inferSchema=True)

# Exiba o DataFrame
df_from_s3.show()

+---------+--------+-------+---+
|firstname|lastname|country| id|
+---------+--------+-------+---+
|  Michael|    Rose|    USA|  2|
|    James|   Smith|    USA|  1|
+---------+--------+-------+---+



In [13]:
# df.write.format("delta").mode("overwrite").save(s3_path)


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 40074)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =